Author: Jianji Chen

email: jianjichen001@gmail.com

Please feel free to reach out with any questions or comments. Thank you!

Note: If you would like to replicate the results of the following codes, please make sure that you have alreay followed the previous process as recorded in the Jupyter Notebook files "clean_demographics" and "clean_job_history".

In [ ]:
import pandas as pd
import numpy as np

In [33]:
demographics = pd.read_csv('data_out/demographics.csv')

In [39]:
job_history = pd.read_csv('data_out/job_history.csv')

In [125]:
merge_long = pd.merge(job_history, demographics.drop(["birth_year", "gender"], axis = 1), on = "mergeid", how = "left")
# job_history does not contain missing values of "birth_year" and "gender",
# and the final sample will be based on job_history for job information (2 waves of information compared to 8 waves of demographics)

In [126]:
merge_long.migrant_year = merge_long.migrant_year.case_when(
    [
        (merge_long.migrant_year == "Don't know", np.nan),
        (merge_long.migrant_year == "Refusal", np.nan)
    ]
)

merge_long["migrant_age"] = merge_long["migrant_year"].astype(float) - merge_long["birth_year"]

In [127]:
merge_long.citizenship_year = merge_long.citizenship_year.case_when(
    [
        (merge_long.citizenship_year == "Don't know", np.nan),
        (merge_long.citizenship_year == "Refusal", np.nan)
    ]
)

merge_long["citizenship_age"] = merge_long["citizenship_year"].astype(float) - merge_long["birth_year"]

In [128]:
merge_long["education_tertiary"] = merge_long["education_level"].case_when(
    [
        (merge_long["education_level"] != "Tertiary education", "Lower education")
    ]
)

In [140]:
# for each respondent with all person-age observations, check if they had ever been in occupational positions in "manager" levels according isco 08 categories
# for those ever been in such positions, find the age when first in such positions

# Define a function to count occurrences of the target value
def count_target(series, target_val):
    return True if (series == target_val).sum() > 0 else False

# Group by 'mergeid' and apply aggregation
history_manager = job_history.groupby('mergeid').agg(
    # Count rows where "occupation_level" equals "Manager"
    manager_ever = ("occupation_level", lambda x: count_target(x, "Manager")),
    # Get the first row where "occupation_level" equals "Manager" (using iloc[0] for safety)
    # first_row_with_target=("occupation_level", lambda x: x[x == "Manager"].iloc[0] if (x == "Manager").any() else None),
    # Get the age from the first row that matches the condition
    manager_1stage = ("age", lambda x: x[job_history.loc[x.index, "occupation_level"] == "Manager"].iloc[0] if (job_history.loc[x.index, "occupation_level"] == "Manager").any() else None)
)

for i in range(1, 60):
    history_manager[f'manager_ever_age{i}'] = (i >= history_manager["manager_1stage"])

# print(history_manager)

In [ ]:
# convert the history_manager to long formart dataframe
history_manager_long = pd.melt(history_manager.reset_index(),
                                            id_vars = 'mergeid',
                                            value_vars = [c for c in list(history_manager) if c.startswith("manager_ever_age")],
                                            var_name = 'age',
                                            value_name = 'manager_ever_cumulative')

history_manager_long["age"] = history_manager_long["age"].str.replace("manager_ever_age", "").astype(int)
history_manager_long["manager_ever_cumulative"] = history_manager_long["manager_ever_cumulative"].astype(int)

In [61]:
# for each respondent with all person-age observations, check if they had ever been in occupational positions in "manager" or "professional" levels according isco 08 categories
# for those ever been in such positions, find the age when first in such positions

# Define a function to count occurrences of the target value
def count_target(series, target_val):
    return True if (series == target_val).sum() > 0 else False

# Group by 'mergeid' and apply aggregation
history_manager_professional = job_history.groupby('mergeid').agg(
    # Count rows where "occupation_level_merge" equals "Manager/professional"
    manager_professional_ever = ("occupation_level_merge", lambda x: count_target(x, "Manager/professional")),
    # Get the first row where "occupation_level_merge" equals "Manager/professional" (using iloc[0] for safety)
    # first_row_with_target=("occupation_level_merge", lambda x: x[x == "Manager/professional"].iloc[0] if (x == "Manager/professional").any() else None),
    # Get the age from the first row that matches the condition
    manager_professional_1stage = ("age", lambda x: x[job_history.loc[x.index, "occupation_level_merge"] == "Manager/professional"].iloc[0] if (job_history.loc[x.index, "occupation_level_merge"] == "Manager/professional").any() else None)
)

for i in range(1, 61):
    history_manager_professional[f'manager_professional_ever_age{i}'] = (i >= history_manager_professional["manager_professional_1stage"])

# print(history_manager_professional)

In [ ]:
# convert the history_manager_professional to long formart dataframe
history_manager_professional_long = pd.melt(history_manager_professional.reset_index(),
                                            id_vars = 'mergeid',
                                            value_vars = [c for c in list(history_manager_professional) if c.startswith("manager_professional_ever_age")],
                                            var_name = 'age',
                                            value_name = 'manager_professional_ever_cumulative')

history_manager_professional_long["age"] = history_manager_professional_long["age"].str.replace("manager_professional_ever_age", "").astype(int)
history_manager_professional_long["manager_professional_ever_cumulative"] = history_manager_professional_long["manager_professional_ever_cumulative"].astype(int)

In [ ]:
# consider to exclude some countires of interview


# exclude Israel and 11 Eastern European countries of interview due to complicated geopolitical and immigration histories during the past decades
# eg., Soviet Union and Former Yugoslavia
# These countries are: Bulgaria, Croatia, Czech Republic, Estonia, Hungary, Lithuania, Latvia, Poland, Romania, Slovenia, Slovakia
countries_filter1 = [
    "Austria", "Belgium", "Cyprus", "Denmark",
    "Finland", "France", "Germany", "Greece",
    "Ireland", "Italy", "Luxembourg", "Malta", 
    "Netherlands", "Portugal", "Spain", "Sweden", "Switzerland"
]


# further exclude 5 countries of interview that have too few non-native respondents
# conditions: less than 20 non-European immigrants or less than 20 European immigrants
# these countries are: Cyprus, Finland, Ireland (too few non-European immigrants), Malta, Portugal, 

countries_filter2 = [
    "Austria", "Belgium", "Denmark", "France",
    "Germany", "Greece", "Italy", "Luxembourg",
    "Netherlands", "Spain", "Sweden", "Switzerland"
]

pd.crosstab(merge_long.loc[merge_long["age"] == 40, "country"],
            merge_long.loc[merge_long["age"] == 40, "born_same_country"],
            dropna=True, normalize=False, margins=True)

In [147]:
# merge a long dataframe that contains demographic information and job situation by age

data_long = pd.merge(merge_long[["mergeid", "age", "country", "gender", "education_tertiary",
                                 "native_or_immigrant", "born_same_country", "migrant_age",
                                 "own_citizenship", "citizenship_age", "occupation_level"]],
                                 history_manager_long,
                                 on = ["mergeid", "age"], how = "inner")

data_long = pd.merge(data_long,
                     history_manager_professional_long,
                     on = ["mergeid", "age"], how = "inner")


data_long["age"] = data_long["age"].astype(int)

# subset this long dataframe that excludes some coutries of interview and people who immigranted after age 45
# also remove the observations where age is younger than 21 or older than 60
data_long_subset = data_long[data_long["country"].isin(countries_filter2)]
data_long_subset = data_long_subset[(data_long_subset["migrant_age"] < 46) | (data_long_subset["migrant_age"].isna())]
data_long_subset = data_long_subset[data_long_subset["age"] > 20] # the age limit was alredy set to 60 earlier

In [133]:
# merge a wide dataframe that contains demographic information and whether a respondent had ever been
# in manager or professional position and the age when first in such a position

data_wide = pd.merge(merge_long[["mergeid", "country", "gender", "education_tertiary",
                                 "native_or_immigrant", "born_same_country", "migrant_age",
                                 "own_citizenship", "citizenship_age"]].groupby("mergeid").first(),
                     history_manager[["manager_ever", "manager_1stage"]],
                     on = "mergeid", how = "right")

data_wide = pd.merge(data_wide,
                     history_manager_professional[["manager_professional_ever", "manager_professional_1stage"]],
                     on = "mergeid", how = "left")

data_wide = data_wide.reset_index()


# subset this wide dataframe that excludes some coutries of interview and people who immigranted after age 45
data_wide_subset = data_wide[data_wide["country"].isin(countries_filter2)]
data_wide_subset = data_wide_subset[(data_wide_subset["migrant_age"] < 46) | (data_wide_subset["migrant_age"].isna())]

In [148]:
## save the data
data_wide.to_csv('data_out/data_wide.csv', index = False)
data_wide_subset.to_csv('data_out/data_wide_subset.csv', index = False)
data_long.to_csv('data_out/data_long.csv', index = False)
data_long_subset.to_csv('data_out/data_long_subset.csv', index = False)